In [ ]:
import os
import pandas as pd 
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

# Definindo o modelo com arquitetura Squeeze Net

In [ ]:
import keras
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization
from keras.layers import MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD

bnmomemtum=0.9

def fire(x, squeeze, expand):
    y  = Conv2D(filters=squeeze, kernel_size=1, activation='relu', padding='same')(x)
    y = BatchNormalization(momentum=bnmomemtum)(y)
    y1 = Conv2D(filters=expand//2, kernel_size=1, activation='relu', padding='same')(y)
    y1 = BatchNormalization(momentum=bnmomemtum)(y1)
    y3 = Conv2D(filters=expand//2, kernel_size=3, activation='relu', padding='same')(y)
    y3 = BatchNormalization(momentum=bnmomemtum)(y3)
    return keras.layers.concatenate([y1, y3])

def fire_module(squeeze, expand):
    return lambda x: fire(x, squeeze, expand)

def get_model(num_classes):
    x = keras.layers.Input(shape=(192, 192, 3))

    y = Conv2D(kernel_size=3, filters=32, padding='same', use_bias=True, activation='relu')(x)
    y = BatchNormalization(momentum=bnmomemtum)(y)
    y = fire_module(24, 48)(y)
    y = MaxPooling2D(pool_size=2)(y)
    y = fire_module(48, 96)(y)
    y = MaxPooling2D(pool_size=2)(y)
    y = fire_module(64, 128)(y)
    y = MaxPooling2D(pool_size=2)(y)
    y = fire_module(48, 96)(y)
    y = MaxPooling2D(pool_size=2)(y)
    y = fire_module(24, 48)(y)
    y = GlobalAveragePooling2D()(y)
    y = Dense(num_classes, activation='softmax')(y)

    return keras.Model(x, y)


# Definindo classe responsável por executar o treinamento com o modelo

In [ ]:
import os
from matplotlib.pyplot import cla
import numpy as np
import keras
from keras.preprocessing import image
from keras.optimizers import SGD
from matplotlib import pyplot as plt

class AIModule:
    def __init__(self, batch_size=32, epochs=5):
        self.batch_size = batch_size
        self.epochs = epochs
        self.IMG_HEIGHT = 192
        self.IMG_WIDTH = 192

    def train(self, classes, model_name, train_generator, validation_generator):

        # Model
        self.model = get_model(len(classes))
        self.model.compile(optimizer=SGD(lr=0.001, momentum=0.9),
                           loss='categorical_crossentropy', metrics=['accuracy'])

        img_width, img_height = self.IMG_HEIGHT, self.IMG_WIDTH

        epochs = self.epochs
        batch_size = self.batch_size
      
        hist = self.model.fit(
            train_generator,
            steps_per_epoch=len(train_generator),
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=len(validation_generator),
            verbose=2)

        self.model.save(model_name)

        # plot loss
        plt.subplot(211)
        plt.title('Cross Entropy Loss')
        plt.plot(hist.history['loss'], color='blue', label='train')
        plt.plot(hist.history['val_loss'], color='orange', label='test')
        # plot accuracy
        plt.subplot(212)
        plt.title('Classification Accuracy')
        plt.plot(hist.history['accuracy'], color='blue', label='train')
        plt.plot(hist.history['val_accuracy'], color='orange', label='test')

        plt.savefig(model_name + '_statistics.png')
        plt.close()
        
        return self.model

# Preparando os dados

In [ ]:
filenames = os.listdir("./train/train")
categories = []

for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})

In [ ]:
df["category"] = df["category"].replace({0: 'cat', 1: 'dog'}) 

In [ ]:
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=10

print('Qtd. Validação: ', total_validate)
print('Qtd. Treino: ', total_train)

# Utilizando Generators para fazer Data Augmentation

In [ ]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "./train/train/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)


validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "./train/train/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

# Fit Model

In [ ]:
ai_module = AIModule()

model = ai_module.train([ 'cat', 'dog' ], 'model.h5', train_generator, validation_generator)

# Prepara os dados para teste

In [ ]:
test_filenames = os.listdir("./test1/test1")
test_df = pd.DataFrame({
    'filename': test_filenames
})
nb_samples = test_df.shape[0]

test_gen = ImageDataGenerator(rescale=1./255)
test_generator = test_gen.flow_from_dataframe(
    test_df, 
    "./test1/test1/", 
    x_col='filename',
    y_col=None,
    class_mode=None,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    shuffle=False
)

# Realiza o predict e formata os resultados

In [ ]:
predict = model.predict_generator(test_generator, steps=np.ceil(nb_samples/batch_size))

#Verifica qual categoria possui mais possibilidade de ser a correta
test_df['category'] = np.argmax(predict, axis=-1)

label_map = dict((v,k) for k,v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)
test_df['category'] = test_df['category'].replace({ 'dog': 1, 'cat': 0 })

sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize=(12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img("../test1/test1/"+filename, target_size=IMAGE_SIZE)
    plt.subplot(6, 3, index+1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')' )
plt.tight_layout()
plt.show()